# Recursive Clustering and Summarization

Plan:
- recursively cluster collections
- create tree of clusters
- cluster until max-depth is reached or (better) until each leaf only has one "plausible" cluster (based on thresholds or probabilities)
- try summarizing to get "main idea" out of cluster


## Recursively cluster 

Based on the topic_clustering notebook, we will try with Agglomerative Clustering 

In [3]:
import pandas as pd

df = pd.read_csv("downloads/40k_balanced_pm_acl.csv")#.sample(frac=0.5)

In [10]:
pos = df[df.labels == 1]
sentences = list(pos["text"]) #otherwise key error

In [11]:
from sentence_transformers import SentenceTransformer, util

print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/342 [00:00<?, ?it/s]

In [18]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

def cluster(embeddings, **kwargs):
    embeddings = embeddings.cpu()
    # Normalize the embeddings to unit length
    corpus_embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Perform kmean clustering
    clustering_model = AgglomerativeClustering(**kwargs) #, affinity='cosine', linkage='average', distance_threshold=0.4)
    clustering_model.fit(corpus_embeddings)
   # cluster_assignment = clustering_model.labels_
    return clustering_model

In [16]:
def print_clusters(clustering_model):
    
    clustered_sentences = {}
    for sentence_id, cluster_id in enumerate(clustering_model.labels_):
        if cluster_id not in clustered_sentences:
            clustered_sentences[cluster_id] = []
        clustered_sentences[cluster_id].append(sentences[sentence_id])
    
    
    for i, cluster in clustered_sentences.items():
        print("Cluster ", i+1)
        print(cluster)
        print("\n")

In [20]:
cluster_model = cluster(embeddings[:1000],n_clusters=None, distance_threshold=0.8)

In [22]:
cluster_model??

Type:        AgglomerativeClustering
String form: AgglomerativeClustering(distance_threshold=0.8, n_clusters=None)
File:        ~/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/sklearn/cluster/_agglomerative.py
Source:     
class AgglomerativeClustering(ClusterMixin, BaseEstimator):
    """
    Agglomerative Clustering

    Recursively merges the pair of clusters that minimally increases
    a given linkage distance.

    Read more in the :ref:`User Guide <hierarchical_clustering>`.

    Parameters
    ----------
    n_clusters : int or None, default=2
        The number of clusters to find. It must be ``None`` if
        ``distance_threshold`` is not ``None``.

    affinity : str or callable, default='euclidean'
        Metric used to compute the linkage. Can be "euclidean", "l1", "l2",
        "manhattan", "cosine", or "precomputed".
        If linkage is "ward", only "euclidean" is accepted.
        If "precomputed", a distance matrix (instead of a similarity matrix)
  

In [ ]:
for cluster in clusters:
    